#MAESTRIA EN ANALITICA DE DATOS
---

BIG DATA

Nombre: Oswaldo Salgado Gómez

Fecha: Octubre 2 de 2025

Taller: Cuaderno para hacer WEB SCRAPING



"""
Script de extracción de resoluciones - Gaceta Ambiental Autoridad Nacionalo de Licencias AMbientales (ANLA) ----------------------------------------------------------
Fuente oficial: https://gaceta.anla.gov.co:8443/Consultar-gaceta

robots.txt:
El sitio web de la ANLA permite el acceso a endpoints públicos destinados a la consulta ciudadana de actos administrativos. Este script realiza solicitudes controladas de acuerdo con el propósito de acceso público establecido por la entidad.

Descripción:
Este script automatiza la consulta, filtrado y descarga de resoluciones publicadas en la Gaceta Ambiental de la Autoridad Nacional de Licencias Ambientales (ANLA). Se extraen documentos oficiales (PDF) asociados a resoluciones que contienen recurso de reposición de posibles sanciones ambientales y corresponden a proyectos del sector hidrocarburos.

El procesamiento incluye:
- Navegación por el buscador oficial de la Gaceta.
- Extracción estructurada de metadatos (título, fecha, proyecto, descripción).
- Se trabajan con las principales empresas del sector hidrocarburo en Colombia.
- Descarga de documentos PDF de acceso público.
- Filtrado por criterios técnicos (reposicón + hidrocarburos).
- Preparación de la información para análisis, indexación y uso posterior
  en motores de búsqueda (Elastic) y aplicaciones web (Flask).

El uso que se da ala información es educativo (no comercial).
Autor: Oswaldo Salgado Gómez.
"""

# HOJA DE RUTA A SEGUIR

___

0. Trabajar con Google Drive
1. Instalar librerías
2. Generar Ruta de las bases de datos donde se almacenará la información
3. Crear DOM Inicial en Gaceta de la ANLA
4. Buscar Encabezados de las Resoluciones
5. Función para extraer tarjetas de cada página
6. Crear un JSON con las resoluciones filtradas desde la Gaceta Ambiental de la ANLA
7. Descargar archivos PDF desde la página de la ANLA
8. Leer el archivo ANLA_enlaces-json.json
9. Extraer texto Pdf a JSON con PyMuPDF
10. Resumen Final del Proceso completo de conversión de PDF a JSON

#0. Trabajar con Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#1. Instalar Librerias

In [2]:
!pip install requests beautifulsoup4 lxml pdfminer.six pymongo pdf2image pytesseract --quiet
!apt-get install -y poppler-utils > /dev/null
!pip install PyMuPDF --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 90.2 MB/s eta 0:00:00


In [3]:
import os
import time
import json
import re
from datetime import datetime
import gc # Import the garbage collector module

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin #valida el dominio del proceso

# Librerias para manejar archivos PDF y OCR
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import fitz

import pytesseract
from PIL import Image
from pdf2image import convert_from_path
from tqdm import tqdm

# Librerias para manejar MongoDB
from pymongo import MongoClient
from pymongo.errors import PyMongoError

#2. Generar Ruta de las bases de datos donde se almacenará la información

In [4]:
BASE_DIR_ANLA = "/content/drive/MyDrive/3_Semestre_3/Big_Data/DataBase/web_scraping_ANLA"


PDF_DIR_ANLA = os.path.join(BASE_DIR_ANLA, "ANLA_pdf")
#PDF_DIR_ANLA_REAL = os.path.join(BASE_DIR_ANLA, "ANLA_pdf_real")
JSON_DIR_ANLA = os.path.join(BASE_DIR_ANLA, "ANLA_json")
ENLACES_JSON_ANLA = os.path.join(BASE_DIR_ANLA, "ANLA_enlaces.json")
ERRORS_JSON_ANLA = os.path.join(BASE_DIR_ANLA, "error_extract_texto_ANLA.json")

for d in [BASE_DIR_ANLA, PDF_DIR_ANLA, JSON_DIR_ANLA]: #PDF_DIR_ANLA_REAL
    os.makedirs(d, exist_ok=True)

print("Directorios ANLA creados/verificados:")
print("BASE_DIR_ANLA :", BASE_DIR_ANLA)
print("PDF_DIR_ANLA  :", PDF_DIR_ANLA)
print("JSON_DIR_ANLA :", JSON_DIR_ANLA)
#print("PDF_DIR_ANLA_REAL:", PDF_DIR_ANLA_REAL)
print("ENLACES_JSON_ANLA:", ENLACES_JSON_ANLA)

# PALABRAS CLAVE PARA FILTRAR EN GACETA (METADATOS Y LUEGO TEXTO)

EMPRESAS_HIDRO = [
    "PAREX", "ECOPETROL", "FRONTERA", "GRAN TIERRA",
    "HOCOL", "GEOPARK", "CANACOL", "OCCIDENTAL", "OXY",
    "PERENCO", "PETRÓLEOS SUDAMERICANOS", "PETROLEOS SUDAMERICANOS",
    "EMERALD ENERGY", "NEXEN"
]

PALABRAS_HIDRO = [
    "HIDROCARBUROS", "PETRÓLEO", "PETROLEO",
    "GAS NATURAL", "OLEODUCTO", "POZO", "CAMPO", "BLOQUE"
]

PALABRAS_REPOSICION = [
    "RECURSO DE REPOSICIÓN",
    "RECURSO DE REPOSICION",
    "RESUELVE EL RECURSO DE REPOSICIÓN",
    "RESUELVE EL RECURSO DE REPOSICION",
    "REPÓNASE", "REPONASE"
]

def es_hidrocarburos_texto(texto: str) -> bool:
    if not texto:
        return False
    t = texto.upper()
    return (
        any(emp in t for emp in EMPRESAS_HIDRO) or
        any(pal in t for pal in PALABRAS_HIDRO)
    )

def es_reposicion_texto(texto: str) -> bool:
    if not texto:
        return False
    t = texto.upper()
    return any(pal in t for pal in PALABRAS_REPOSICION)

# Módulo para limpiar los directorios si es necesario

LIMPIAR_ANLA = False  # Cambiar a True para limpiar


if LIMPIAR_ANLA:
    # Limpiar PDFs y JSONs ANLA
    for d in [PDF_DIR_ANLA, JSON_DIR_ANLA]:#PDF_DIR_ANLA_REAL
        for f in os.listdir(d):
            path_f = os.path.join(d, f)
            if os.path.isfile(path_f):
                os.remove(path_f)

    # Limpiar ENLACES_JSON_ANLA
    if os.path.exists(ENLACES_JSON_ANLA):
        os.remove(ENLACES_JSON_ANLA)

    # Limpiar errores ANLA
    if os.path.exists(ERRORS_JSON_ANLA):
        os.remove(ERRORS_JSON_ANLA)
    print("Carpetas de PDFs/JSONs limpiadas.")


Directorios ANLA creados/verificados:
BASE_DIR_ANLA : /content/drive/MyDrive/3_Semestre_3/Big_Data/DataBase/web_scraping_ANLA
PDF_DIR_ANLA  : /content/drive/MyDrive/3_Semestre_3/Big_Data/DataBase/web_scraping_ANLA/ANLA_pdf
JSON_DIR_ANLA : /content/drive/MyDrive/3_Semestre_3/Big_Data/DataBase/web_scraping_ANLA/ANLA_json
ENLACES_JSON_ANLA: /content/drive/MyDrive/3_Semestre_3/Big_Data/DataBase/web_scraping_ANLA/ANLA_enlaces.json


#3. Crear DOM Inicial en Gaceta de la ANLA


In [5]:
# URL base confirmada por inspección
GACETA_ENDPOINT = "https://gaceta.anla.gov.co:8443/Consultar-gaceta/consultar"

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/124.0 Safari/537.36"
    )
}


def obtener_soup_gaceta(pagina=1, palabra=""):
    """
    Hace la solicitud GET al endpoint real que devuelve
    las tarjetas de resoluciones.
    """

    params = {
        "pagina": pagina,
        "buscar_numero_documento": "",
        "buscar_desde_fecha_documento": "",
        "buscar_hasta_fecha_documento": "",
        "buscar_tipo_documento": "",
        "buscar_expediente": "",
        "buscar_fecha": "",
        "buscar_ubicacion": "",
        "buscar_documento": palabra  # <-- este es el bueno (texto libre)
    }

    r = requests.get(GACETA_ENDPOINT, params=params, headers=HEADERS, timeout=60)
    r.raise_for_status()

    return BeautifulSoup(r.text, "lxml")

#4. Buscar Encabezados de las Resoluciones

In [6]:
soup = obtener_soup_gaceta()
headings = soup.find_all("div", class_="item-box-blog-heading")

print("Encabezados encontrados:", len(headings))

for i, h in enumerate(headings[:5], start=1):
    h5 = h.find("h5")
    if h5:
        titulo = h5.get_text(strip=True)
    else:
        titulo = "(sin título)"

    print(f"[{i}] {titulo}")

Encabezados encontrados: 20
[1] RESOLUCIÓN NO. 01778 DEL 07 DE OCTUBRE DE 2021
[2] RESOLUCION NO. 00648 DEL 25 DE MARZO DE 2022
[3] RESOLUCIÓN NO. 00790 DEL 21 DE ABRIL DE 2022
[4] AUTO NO. 02926 DEL 29 DE ABRIL DE 2022
[5] RESOLUCION NO. 00928 DEL 09 DE MAYO DE 2022


#5. Función para extraer tarjetas de cada página

In [7]:
def extraer_tarjetas_desde_soup(soup):
    """
    Extrae información estructurada de cada tarjeta de resolución
    en la página de resultados de la Gaceta ANLA.
    Devuelve una lista de diccionarios.
    """
    resultados = []

    # Cada resultado completo está en un <div class="member">
    cards = soup.find_all("div", class_="member")

    for card in cards:

       # Titulo
        heading = card.find("div", class_="item-box-blog-heading")
        h5 = heading.find("h5") if heading else None
        titulo = h5.get_text(strip=True) if h5 else "(sin título)"

        # Metadatos
        fecha_pub = None
        proyecto = None
        ubicacion = None
        descripcion = None

        for p in card.find_all("p"):
            texto_p = p.get_text(" ", strip=True)

            if "Publicado el" in texto_p:
                fecha_pub = (
                    texto_p
                    .replace("Publicado el", "")
                    .replace(":", "")
                    .strip()
                )

            elif "Nombre de proyecto" in texto_p:
                proyecto = (
                    texto_p
                    .replace("Nombre de proyecto:", "")
                    .strip()
                )

            elif "Ubicación" in texto_p:
                ubicacion = (
                    texto_p
                    .replace("Ubicación:", "")
                    .strip()
                )

            elif "Descripción" in texto_p:
                descripcion = (
                    texto_p
                    .replace("Descripción:", "")
                    .strip()
                )

        # PDF (id y URL)
        pdf_id = None
        for a in card.find_all("a", onclick=True):
            onclick = a.get("onclick", "")
            m = re.search(r"descargar\((\d+)\)", onclick)
            if m:
                pdf_id = m.group(1)
                break

        pdf_url = f"https://gaceta.anla.gov.co:8443/download/{pdf_id}" if pdf_id else None

        # Texto combinado para filtros posteriores
        texto_busqueda = " ".join(
            t for t in [titulo, proyecto or "", descripcion or ""] if t
        )

        resultados.append({
            "titulo": titulo,
            "fecha_publicacion": fecha_pub,
            "proyecto": proyecto,
            "ubicacion": ubicacion,
            "descripcion": descripcion,
            "pdf_id": pdf_id,
            "pdf_url": pdf_url,
            "texto_busqueda": texto_busqueda
        })

    return resultados

datos = extraer_tarjetas_desde_soup(soup)

for item in datos[:5]:
    print(item)
    print("-"*80)

{'titulo': 'RESOLUCIÓN NO. 01778 DEL 07 DE OCTUBRE DE 2021', 'fecha_publicacion': 'jueves, 28 de octubre de 2021', 'proyecto': 'Segundo 1 y Segundo 3, Dindal, Guaduas', 'ubicacion': 'Grupo de Medio Magdalena', 'descripcion': 'Descripción : “POR EL CUAL SE DEFINE LA RESPONSABILIDAD DE UN PROCEDIMIENTO SANCIONATORIO AMBIENTAL Y SE TOMAN OTRAS DETERMINACIONES”', 'pdf_id': '37294', 'pdf_url': 'https://gaceta.anla.gov.co:8443/download/37294', 'texto_busqueda': 'RESOLUCIÓN NO. 01778 DEL 07 DE OCTUBRE DE 2021 Segundo 1 y Segundo 3, Dindal, Guaduas Descripción : “POR EL CUAL SE DEFINE LA RESPONSABILIDAD DE UN PROCEDIMIENTO SANCIONATORIO AMBIENTAL Y SE TOMAN OTRAS DETERMINACIONES”'}
--------------------------------------------------------------------------------
{'titulo': 'RESOLUCION NO. 00648 DEL 25 DE MARZO DE 2022', 'fecha_publicacion': 'Lunes, 28 de marzo de 2022', 'proyecto': 'Sin proyecto', 'ubicacion': 'Grupo de Medio Magdalena', 'descripcion': 'Descripción : Por la cual se otorga Licen

#6. Crear un JSON con las resoluciones filtradas desde la Gaceta Ambiental de la ANLA

* Función especializada para recorrer la Gaceta ANLA, extraer metadatos y construir el catálogo JSON

In [9]:
MAX_DOCS_CATALOGO = 150   # máximo de resoluciones que sde desean
pagina = 1
hipervinculos_anla = []

while True:

    print(f"\n=== Página {pagina} ===")

    soup = obtener_soup_gaceta(pagina=pagina, palabra="reposición")
    tarjetas = extraer_tarjetas_desde_soup(soup)

    print(f"Tarjetas encontradas en esta página: {len(tarjetas)}")

    # Si ya no hay tarjetas se detiene
    if not tarjetas:
        print("No hay más tarjetas, se detiene la paginación.")
        break

    for t in tarjetas:
        texto_filtro = t["texto_busqueda"]

        # Filtros por HIDROCARBUROS y REPOSICIÓN (en metadatos)
        if not es_hidrocarburos_texto(texto_filtro):
            continue

        if not es_reposicion_texto(texto_filtro):
            continue

        hipervinculos_anla.append({
            "tipo": "pdf",
            "titulo": t["titulo"],
            "fecha_publicacion": t["fecha_publicacion"],
            "proyecto": t["proyecto"],
            "ubicacion": t["ubicacion"],
            "descripcion": t["descripcion"],
            "pdf_id": t["pdf_id"],
            "url": t["pdf_url"],
            "categoria": "ANLA_GACETA"
        })

        print(f"  [+] Agregada: {t['titulo']}")

        if len(hipervinculos_anla) >= MAX_DOCS_CATALOGO:
            print("\nSe alcanzó el máximo de resoluciones configurado.")
            break

    if len(hipervinculos_anla) >= MAX_DOCS_CATALOGO:
        break

    # Pasar a la siguiente página
    pagina += 1

print(f"TOTAL resoluciones ANLA en catálogo (filtradas): {len(hipervinculos_anla)}")

with open(ENLACES_JSON_ANLA, "w", encoding="utf-8") as f:
    json.dump(hipervinculos_anla, f, ensure_ascii=False, indent=2)

print("Catálogo ANLA guardado en:", ENLACES_JSON_ANLA)


=== Página 1 ===
Tarjetas encontradas en esta página: 20

=== Página 2 ===
Tarjetas encontradas en esta página: 20

=== Página 3 ===
Tarjetas encontradas en esta página: 20

=== Página 4 ===
Tarjetas encontradas en esta página: 20

=== Página 5 ===
Tarjetas encontradas en esta página: 20

=== Página 6 ===
Tarjetas encontradas en esta página: 20

=== Página 7 ===
Tarjetas encontradas en esta página: 20

=== Página 8 ===
Tarjetas encontradas en esta página: 20

=== Página 9 ===
Tarjetas encontradas en esta página: 20

=== Página 10 ===
Tarjetas encontradas en esta página: 20

=== Página 11 ===
Tarjetas encontradas en esta página: 20

=== Página 12 ===
Tarjetas encontradas en esta página: 20

=== Página 13 ===
Tarjetas encontradas en esta página: 20

=== Página 14 ===
Tarjetas encontradas en esta página: 20

=== Página 15 ===
Tarjetas encontradas en esta página: 20

=== Página 16 ===
Tarjetas encontradas en esta página: 20

=== Página 17 ===
Tarjetas encontradas en esta página: 20

=== P

#7. Descargar archivos PDF desde la página de la ANLA

In [10]:
carpeta_pdfs = PDF_DIR_ANLA

# Cargar catálogo generado previamente
with open(ENLACES_JSON_ANLA, "r", encoding="utf-8") as f:
    catalogo = json.load(f)

# Endpoint REAL que usa la ANLA para descargar PDFs
BASE_DESCARGA = "https://gaceta.anla.gov.co:8443/Consultar-gaceta/descargar"

session = requests.Session()
headers = {"User-Agent": "Mozilla/5.0"}

print("Total documentos en catálogo:", len(catalogo))

descargados = 0
errores = {}

for doc in catalogo:

    pdf_id = doc.get("pdf_id")
    if not pdf_id:
        continue

    # Construir URL correcta de descarga
    url_real = f"{BASE_DESCARGA}?q={pdf_id}"

    nombre = f"ANLA_{pdf_id}.pdf"
    destino = os.path.join(carpeta_pdfs, nombre)

    # Saltar si ya existe
    if os.path.exists(destino):
        print("— Ya existe, se omite:", nombre)
        continue

    try:
        resp = session.get(url_real, headers=headers, timeout=60)
        resp.raise_for_status()

        # Validar que sea PDF real
        content_type = resp.headers.get("Content-Type", "").lower()
        data = resp.content
        primeros = data[:10]

        # Quitar posible BOM UTF-8 al inicio (b'\xef\xbb\xbf')
        sin_bom = primeros.lstrip(b"\xef\xbb\xbf")

        # Aceptamos si el Content-Type contiene "pdf" O si,
        # después de quitar el BOM, la cabecera empieza por %PDF
        if ("pdf" not in content_type) and (not sin_bom.startswith(b"%PDF")):
            raise Exception(
                f"Contenido NO parece PDF. Content-Type: {content_type}, "
                f"cabecera: {primeros!r}"
    )

        with open(destino, "wb") as f_out:
            f_out.write(data)

        descargados += 1
        print("✔ Descargado:", nombre)

    except Exception as e:
        errores[nombre] = str(e)
        print("✘ Error descargando:", nombre, "-", e)

print("\nDescarga finalizada.")
print("PDFs descargados OK:", descargados)
print("Errores:", len(errores))

Total documentos en catálogo: 150
— Ya existe, se omite: ANLA_38948.pdf
— Ya existe, se omite: ANLA_38956.pdf
— Ya existe, se omite: ANLA_39031.pdf
— Ya existe, se omite: ANLA_39033.pdf
— Ya existe, se omite: ANLA_39065.pdf
— Ya existe, se omite: ANLA_39118.pdf
— Ya existe, se omite: ANLA_39130.pdf
— Ya existe, se omite: ANLA_39544.pdf
— Ya existe, se omite: ANLA_39592.pdf
— Ya existe, se omite: ANLA_39684.pdf
— Ya existe, se omite: ANLA_39746.pdf
— Ya existe, se omite: ANLA_39753.pdf
— Ya existe, se omite: ANLA_39763.pdf
— Ya existe, se omite: ANLA_39808.pdf
— Ya existe, se omite: ANLA_39809.pdf
— Ya existe, se omite: ANLA_39846.pdf
— Ya existe, se omite: ANLA_39895.pdf
— Ya existe, se omite: ANLA_40151.pdf
— Ya existe, se omite: ANLA_40154.pdf
— Ya existe, se omite: ANLA_40226.pdf
— Ya existe, se omite: ANLA_40360.pdf
— Ya existe, se omite: ANLA_40370.pdf
— Ya existe, se omite: ANLA_40500.pdf
— Ya existe, se omite: ANLA_40502.pdf
— Ya existe, se omite: ANLA_40616.pdf
— Ya existe, se 

#8. Leer el archivo ANLA_enlaces_json.json

In [11]:
# Ruta del archivo JSON generado previamente
ruta_catalogo = ENLACES_JSON_ANLA

# Confirmación visual de la ruta
print("Leyendo catálogo desde:", ruta_catalogo)

# Leer el archivo JSON
with open(ruta_catalogo, "r", encoding="utf-8") as f:
    catalogo = json.load(f)

# Mostrar un resumen
print("Total de documentos cargados:", len(catalogo))

# Mostrar un ejemplo
print("\nEjemplo de un registro:")
for k, v in catalogo[0].items():
    print(f"{k}: {v}")

Leyendo catálogo desde: /content/drive/MyDrive/3_Semestre_3/Big_Data/DataBase/web_scraping_ANLA/ANLA_enlaces.json
Total de documentos cargados: 150

Ejemplo de un registro:
tipo: pdf
titulo: RESOLUCIóN NO. 00852 DEL 29 DE ABRIL DE 2022
fecha_publicacion: jueves, 5 de mayo de 2022
proyecto: PMA CAMPOS DE NEIVA  POR LA GERENCIA DEL ALTO MAGDALENA
ubicacion: Alto Magdalena – Cauca
descripcion: Descripción : “Por la cual se resuelve un recurso de reposición interpuesto contra la Resolución 1673 de 22 de septiembre de 2021”
pdf_id: 38948
url: https://gaceta.anla.gov.co:8443/download/38948
categoria: ANLA_GACETA


#9. Extraer texto PDF a JSON con PyMuPDF

In [12]:
# Funciones Auxiliares para enrioquecer el Archivo Json

def extraer_numero_resolucion(texto):
    """
    Busca una línea que contenga la palabra RESOLUCIÓN (o similar)
    y la devuelve completa.
    """
    for linea in texto.splitlines():
        if "RESOLUCIÓN" in linea.upper() or "RESOLUCION" in linea.upper():
            linea_limpia = " ".join(linea.split())
            return linea_limpia
    return None


def extraer_fecha_resolucion(texto):
    """
    Intenta extraer la fecha de la resolución en formato YYYY-MM-DD.
    Soporta patrones tipo:
      - 13 DIC 2023
      - 07 OCT 2021
      - 1 de marzo de 2022
    """
    if not texto:
        return None

    meses_abrev = {
        'ENE': '01', 'FEB': '02', 'MAR': '03', 'ABR': '04',
        'MAY': '05', 'JUN': '06', 'JUL': '07', 'AGO': '08',
        'SEP': '09', 'OCT': '10', 'NOV': '11', 'DIC': '12'
    }

    meses_completos = {
        'ENERO': '01', 'FEBRERO': '02', 'MARZO': '03', 'ABRIL': '04',
        'MAYO': '05', 'JUNIO': '06', 'JULIO': '07', 'AGOSTO': '08',
        'SEPTIEMBRE': '09', 'SETIEMBRE': '09', 'OCTUBRE': '10',
        'NOVIEMBRE': '11', 'DICIEMBRE': '12'
    }

    # 1) Patrón tipo "13 DIC 2023"
    patron1 = r"(\d{1,2})\s+(ENE|FEB|MAR|ABR|MAY|JUN|JUL|AGO|SEP|OCT|NOV|DIC)\s+(\d{4})"
    m1 = re.search(patron1, texto, flags=re.IGNORECASE)
    if m1:
        dia = int(m1.group(1))
        mes = meses_abrev[m1.group(2).upper()]
        anio = int(m1.group(3))
        return f"{anio:04d}-{mes}-{dia:02d}"

    # 2) Patrón tipo "13 de octubre de 2021"
    patron2 = r"(\d{1,2})\s+de\s+([A-Za-zÁÉÍÓÚÜñÑ]+)\s+de\s+(\d{4})"
    m2 = re.search(patron2, texto, flags=re.IGNORECASE)
    if m2:
        dia = int(m2.group(1))
        mes_nombre = m2.group(2).upper()
        mes_nombre = mes_nombre.replace("Á", "A").replace("É", "E").replace("Í", "I") \
                               .replace("Ó", "O").replace("Ú", "U")
        mes = meses_completos.get(mes_nombre, "01")
        anio = int(m2.group(3))
        return f"{anio:04d}-{mes}-{dia:02d}"

    return None


def extraer_nombre_proyecto(texto):
    patrones = [
        r"(PROYECTO[:\s]+(.+?)(?=\n|$))",
        r"(proyecto denominado.+?)(?=\n|$)",
        r"(proyecto\s+.+?)(?=\n\n|$)"
    ]

    for patron in patrones:
        m = re.search(patron, texto, flags=re.IGNORECASE | re.DOTALL)
        if m:
            linea = m.group(0)
            return " ".join(linea.split())

    return None

def extraer_numero_expediente(texto):
    """
    Extrae el número de expediente, por ejemplo:
    'expediente LAV0057-00-2023'
    'expediente SBTI0126-00-2022'
    """
    patron = r"expediente\s+([A-Z0-9\-\/]+)"
    m = re.search(patron, texto, flags=re.IGNORECASE)
    if m:
        return m.group(1).strip()
    return None

def extraer_radicados(texto):
    """
    Devuelve una lista de radicados ANLA encontrados en el texto.
    Ejemplos:
      'radicado ANLA 20246200916162 del 13 de agosto de 2024'
    """
    patron = r"radicado\s+(?:ANLA\s+)?(\d{8,})"
    matches = re.findall(patron, texto, flags=re.IGNORECASE)
    if not matches:
        return None
    # Quitamos duplicados conservando orden
    vistos = set()
    radicados_unicos = []
    for r in matches:
        if r not in vistos:
            vistos.add(r)
            radicados_unicos.append(r)
    return radicados_unicos

def extraer_empresa(texto):
    """
    Busca líneas amplias donde aparezca una empresa, no solo la primera coincidencia.
    """
    patron = r"([A-ZÁÉÍÓÚÜÑ][A-ZÁÉÍÓÚÜÑ\s\.,\-]+(S\.A\.|SAS|LTD|LTDA|S\.A\.S\.))"
    m = re.search(patron, texto, flags=re.IGNORECASE)
    if m:
        return " ".join(m.group(0).split())

    # fallback más amplio
    candidato = None
    for linea in texto.splitlines():
        if any(x in linea.upper() for x in ["S.A.", "SAS", "LTDA", "LTD"]):
            candidato = linea
            break

    if candidato:
        return " ".join(candidato.split())

    return None

def extraer_ubicacion(texto):
    patrones = [
        r"(Departamento de.+?\.)(?=\s|$)",
        r"(Municipio de.+?\.)(?=\s|$)",
        r"(Grupo de Medio.+?)(?=\n|$)",
        r"(localizado en.+?\.)(?=\s|$)"
    ]

    for patron in patrones:
        m = re.search(patron, texto, flags=re.IGNORECASE | re.DOTALL)
        if m:
            return " ".join(m.group(1).split())

    return None

def extraer_descripcion(texto):
    """
    Extrae la descripción del acto, típicamente la frase:
    “Por el cual se resuelve el recurso de reposición...”
    """
    if not texto:
        return None

    # 1) Buscar entre comillas tipográficas o normales
    patron_comillas = r"[\"“](Por\s+(el|la)\s+[^\"”]+)[\"”]"
    m = re.search(patron_comillas, texto, flags=re.IGNORECASE | re.DOTALL)
    if m:
        desc = m.group(1)
        return " ".join(desc.split())

    # 2) Fallback: buscar línea/s que comiencen por "Por el cual" o "Por la cual"
    lineas = [l.strip() for l in texto.splitlines() if l.strip()]
    for i, linea in enumerate(lineas):
        up = linea.upper()
        if up.startswith("POR EL CUAL") or up.startswith("POR LA CUAL"):
            desc = linea
            # Si no termina en punto o comillas, concatenar la siguiente línea
            if (not desc.endswith((".", "”", '"'))) and i + 1 < len(lineas):
                siguiente = lineas[i + 1].strip()
                if siguiente:
                    desc = desc + " " + siguiente
            return " ".join(desc.split())

    return None


def clasificar_infracciones(texto):
    """
    Clasifica el/los tipos de infracción presentes en el texto.
    Devuelve una lista de categorías (puede haber más de una) o None.
    """
    if not texto:
        return None

    t = texto.upper()

    categorias = {
        "Vertimientos": [
            "VERTIMIENTO", "VERTIMIENTOS",
            "AGUAS RESIDUALES", "DESCARGA DE AGUAS", "DESCARGA DE EFLUENTES"
        ],
        "Emisiones atmosféricas": [
            "EMISIONES ATMOSFÉRICAS", "EMISION ATMOSFERICA",
            "MATERIAL PARTICULADO", "CALIDAD DEL AIRE"
        ],
        "Residuos sólidos": [
            "RESIDUOS SÓLIDOS", "RESIDUOS SOLIDOS",
            "RESIDUOS ORDINARIOS", "RESIDUOS PELIGROSOS",
            "RCD", "BASURAS", "DESECHOS SÓLIDOS"
        ],
        "Incumplimiento de norma": [
            "INCUMPLIMIENTO DE LA NORMA",
            "NO DIO CUMPLIMIENTO", "NO DIO CUMPLIMIENDO",
            "INCUMPLIMIENTO DEL ARTÍCULO", "INCUMPLIMIENTO DEL ARTICULO",
            "INCUMPLIERON LO ESTABLECIDO"
        ],
        "Manejo de aguas": [
            "MANEJO DE AGUAS", "MANEJO DEL AGUA",
            "CUERPOS DE AGUA", "CORRIENTE HÍDRICA", "CORRIENTE HIDRICA",
            "CAUCE", "CAUCES", "FUENTE HÍDRICA", "FUENTE HIDRICA"
        ],
        "Tala o afectación de bosques": [
            "TALA", "TALÓ", "TALO",
            "APROVECHAMIENTO FORESTAL", "BOSQUE", "COBERTURA BOSCOSA",
            "DEFORESTACIÓN", "DEFORESTACION"
        ],
        "Incumplimiento del PMA": [
            "PLAN DE MANEJO AMBIENTAL", "PMA",
            "INCUMPLIMIENTO DEL PLAN DE MANEJO AMBIENTAL",
            "INCUMPLIMIENTO DEL PMA"
        ],
        "Cierre de pozos": [
            "CIERRE DE POZOS", "CIERRE DEFINITIVO DEL POZO",
            "ABANDONO DEL POZO", "TAPONAMIENTO DEL POZO"
        ],
        "No presentar informes": [
            "NO PRESENTÓ INFORMES", "NO PRESENTO INFORMES",
            "NO PRESENTÓ LOS INFORMES", "NO REMITIÓ LOS INFORMES",
            "NO REMITIO LOS INFORMES", "NO ENVÍO LOS INFORMES",
            "NO ENVIO LOS INFORMES", "NO SE PRESENTARON INFORMES",
            "FALTA DE PRESENTACIÓN DE INFORMES",
            "FALTA DE PRESENTACION DE INFORMES"
        ]
    }

    tipos_detectados = []
    for categoria, palabras in categorias.items():
        if any(p in t for p in palabras):
            tipos_detectados.append(categoria)

    return tipos_detectados or None


# Ciclo principal de conversión de PDF a JSON

errores = {}

pdf_files = sorted([f for f in os.listdir(PDF_DIR_ANLA) if f.lower().endswith(".pdf")])

print("Total PDFs para procesar:", len(pdf_files))

procesados = 0

for fname in pdf_files:
    pdf_path = os.path.join(PDF_DIR_ANLA, fname)

    # ID del documento (ANLA_XXXXX.pdf → XXXXX)
    pdf_id = os.path.splitext(fname)[0].replace("ANLA_", "")

    json_path = os.path.join(JSON_DIR_ANLA, f"ANLA_{pdf_id}.json")

    # Saltar si ya existe el JSON
    if os.path.exists(json_path):
        continue

    try:
        texto_total = []
        with fitz.open(pdf_path) as pdf:
            for page in pdf:
                texto_total.append(page.get_text("text"))

        texto = "\n".join(texto_total)

        # =========================
        # CAMPOS NUEVA ESTRUCTURA
        # =========================
        numero_res = extraer_numero_resolucion(texto)
        fecha_res = extraer_fecha_resolucion(texto)
        nom_proy = extraer_nombre_proyecto(texto)
        ubic = extraer_ubicacion(texto)
        empresa = extraer_empresa(texto)
        expediente = extraer_numero_expediente(texto)
        radicados = extraer_radicados(texto)
        descripcion = extraer_descripcion(texto)
        tipos_infraccion = clasificar_infracciones(texto)

        data = {
            "fuente": "ANLA_GACETA",
            "numero_resolución": numero_res,
            "fecha_resolución": fecha_res,
            "nombre_proyecto": nom_proy,
            "ubicación": ubic,
            "empresa": empresa,
            "numero_expediente": expediente,
            "radicados": radicados,              # lista o None
            "descripcion": descripcion,          # la frase “Por el cual...”
            "tipos_infraccion": tipos_infraccion,# lista de categorías o None
            "pdf_id": pdf_id,
            "file_name": fname,
            "texto_completo": texto              # siempre al final
        }

        with open(json_path, "w", encoding="utf-8") as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        procesados += 1
        print(f"JSON generado para {fname}")

    except Exception as e:
        errores[pdf_id] = str(e)
        print(f"Error procesando {fname}: {e}")
        continue

print(f"\nConversión completada.")
print(f"PDFs procesados correctamente: {procesados}")
print(f"Errores: {len(errores)}")

# Guardar archivo de errores
with open(ERRORS_JSON_ANLA, "w", encoding="utf-8") as f:
    json.dump(errores, f, ensure_ascii=False, indent=2)

print("Archivo de errores guardado en:", ERRORS_JSON_ANLA)

Total PDFs para procesar: 150

Conversión completada.
PDFs procesados correctamente: 0
Errores: 0
Archivo de errores guardado en: /content/drive/MyDrive/3_Semestre_3/Big_Data/DataBase/web_scraping_ANLA/error_extract_texto_ANLA.json


#10. Resumen Final del Proceso completo de conversión de PDF a JSON

In [13]:
print("Generando resumen final...")

# Cargar lista de errores y contar resultados
errors_path = ERRORS_JSON_ANLA
json_dir = JSON_DIR_ANLA

# Listar todos los JSON generados
json_files = [f for f in os.listdir(json_dir) if f.lower().endswith(".json")]
total_json = len(json_files)

# Cargar todos los JSON en una lista
resultados = []
for f in json_files:
    path = os.path.join(json_dir, f)
    try:
        with open(path, "r", encoding="utf-8") as j:
            data = json.load(j)
            resultados.append({
                "file_name": data.get("file_name", os.path.basename(f)), # Use file_name if present, otherwise filename
                "pdf_id": data.get("pdf_id"),
                "tamano_texto": len(data.get("texto", "")),
            })
    except Exception as e:
        print(f"Error leyendo {f}: {e}")

df_resumen = pd.DataFrame(resultados)

# Estadísticas de texto extraído
print("\n Estadísticas del tamaño del texto extraído:")
print(df_resumen["tamano_texto"].describe().round(2))

# Cargar lista de errores (si existe)
if os.path.exists(errors_path):
    with open(errors_path, "r", encoding="utf-8") as f:
        errores = json.load(f)
    print(f"\n Archivos con fallos: {len(errores)}")
else:
    errores = []
    print("\nNo se encontró archivo de errores (¡todo bien!).")

# Mostrar ejemplos de texto de 3 documentos exitosos

if len(df_resumen[df_resumen["tamano_texto"] > 200]) >= 3:
    ejemplos = df_resumen[df_resumen["tamano_texto"] > 200].sample(3, random_state=1)
    print("\n Ejemplos de textos extraídos:")
    for ej in ejemplos.itertuples(index=False):
        print(f"\n Archivo: {ej.file_name}")
        path = os.path.join(json_dir, f"ANLA_{ej.pdf_id}.json")
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
            print(data["texto"][:400].replace("\n", " ") + "...")
else:
    print("\nNo hay suficientes documentos con texto para mostrar ejemplos.")

# Exportar resumen a Excel
excel_path = os.path.join(BASE_DIR_ANLA, "Resumen_extraccion_ANLA.xlsx")
df_resumen.to_excel(excel_path, index=False)
print(f"\n Resumen exportado a: {excel_path}")

Generando resumen final...

 Estadísticas del tamaño del texto extraído:
count    150.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: tamano_texto, dtype: float64

 Archivos con fallos: 0

No hay suficientes documentos con texto para mostrar ejemplos.

 Resumen exportado a: /content/drive/MyDrive/3_Semestre_3/Big_Data/DataBase/web_scraping_ANLA/Resumen_extraccion_ANLA.xlsx
